<a href="https://colab.research.google.com/github/Keerthi9470/Text-Summarization-Project/blob/main/NLP_Text_Summarization_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install flask-ngrok

In [2]:
!pip install flask flask-ngrok spacy
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 25.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
^C


In [4]:
!pip install pyngrok
!ngrok authtoken 2t17p7NnAAms8g3usxQA7Hn4u1Q_5MLx9REeZuQ6wR2WQhajN

  Using cached pyngrok-7.2.3-py3-none-any.whl.metadata (8.7 kB)
Using cached pyngrok-7.2.3-py3-none-any.whl (23 kB)
Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
from flask import Flask, render_template_string, request
from transformers import pipeline
from pyngrok import ngrok
import spacy
from collections import Counter
from heapq import nlargest

app = Flask(__name__)
nlp = spacy.load('en_core_web_sm')

# Initialize the summarization pipeline
summarizer = pipeline("summarization", model="t5-base", tokenizer="t5-base", framework="pt")

HTML_TEMPLATE = """
<!DOCTYPE html>
<html>
<head>
    <title>Text Summarizer</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            text-align: center;
            background-color: #F8F9FA;
            margin: 20px;
        }

        /* Title Styling */
        #titl {
            color: #198754;
            font-size: 36px;
            font-weight: bold;
        }

        /* Section Headings */
        h3 {
            color: #495057;
            font-size: 24px;
        }

        /* Container for Forms */
        .container {
            display: flex;
            justify-content: center;
            gap: 20px;
            margin-top: 20px;
        }

        /* Form Boxes */
        .box {
            width: 45%;
            background: white;
            padding: 20px;
            border-radius: 10px;
            box-shadow: 0px 4px 10px rgba(0, 0, 0, 0.1);
            border: 2px solid #198754;
        }

        /* Textarea Input */
        textarea {
            width: 80%;
            padding: 10px;
            font-size: 16px;
            border: 2px solid #198754;
            border-radius: 5px;
            resize: none;
            outline: none;
        }

        /* Number Input */
        input[type="number"] {
            padding: 10px;
            font-size: 16px;
            border-radius: 5px;
            border: 2px solid #198754;
            outline: none;
        }

        /* Submit Button */
        input[type="submit"] {
            background-color: #198754;
            color: white;
            border: 2px solid #157347;
            padding: 10px 20px;
            border-radius: 5px;
            font-size: 16px;
            cursor: pointer;
            transition: 0.3s;
        }
        input[type="submit"]:hover {
            background-color: #145c32;
        }

        /* Summary Output Box */
        .output {
            margin-top: 30px;
            background: white;
            padding: 20px;
            border-radius: 10px;
            box-shadow: 0px 4px 10px rgba(0, 0, 0, 0.1);
            width: 80%;
            margin-left: auto;
            margin-right: auto;
            border: 2px solid #198754;
        }

        /* Summary Text */
        p {
            font-size: 18px;
            color: #555;
            line-height: 1.6;
        }

        /* Responsive Design */
        @media (max-width: 768px) {
            .container {
                flex-direction: column;
                align-items: center;
            }
            .box {
                width: 80%;
            }
        }

    </style>
</head>

<body>
    <h1 id="titl">Text Summarizer</h1>

    <div class="container">
        <div class="box">
            <h3 class="sub">Extractive Summarization</h3>
            <form action="/ext_summarize" method="post">
                <textarea name="text" rows="8" placeholder="Enter text for extractive summarization..."></textarea><br>
                <label>Number of sentences:</label>
                <input type="number" name="num_sentences" value="2" min="1"><br><br>
                <input type="submit" value="Summarize">
            </form>
        </div>

        <div class="box">
            <h3 class="sub">Abstractive Summarization</h3>
            <form action="/abs_summarize" method="post">
                <textarea name="text" rows="8" placeholder="Enter text for abstractive summarization..."></textarea><br><br>
                <input type="submit" value="Summarize">
            </form>
        </div>
    </div>

    {% if summary %}
    <div class="output">
        <h3>Summarized Text:</h3>
        <p>{{ summary }}</p>
    </div>
    {% endif %}
</body>
</html>
"""

@app.route('/')
def home():
    return render_template_string(HTML_TEMPLATE)

@app.route('/ext_summarize', methods=['POST'])
def ext_summarize():
    text = request.form['text']
    num_sentences = int(request.form['num_sentences'])

    if not text.strip():
        return render_template_string(HTML_TEMPLATE, summary="Error: No text provided.")

    doc = nlp(text)
    tokens = [token.text.lower() for token in doc if not token.is_stop and not token.is_punct and token.text != '\n']
    word_freq = Counter(tokens)

    if not word_freq:
        return render_template_string(HTML_TEMPLATE, summary="Error: No valid words found in text.")

    max_freq = max(word_freq.values())
    for word in word_freq.keys():
        word_freq[word] = word_freq[word] / max_freq

    sent_token = [sent.text for sent in doc.sents]
    sent_score = {}
    for sent in sent_token:
        for word in sent.split():
            if word.lower() in word_freq.keys():
                sent_score[sent] = sent_score.get(sent, 0) + word_freq[word]

    summarized_sentences = nlargest(num_sentences, sent_score, key=sent_score.get)
    summary = " ".join(summarized_sentences)

    return render_template_string(HTML_TEMPLATE, summary=summary)

@app.route('/abs_summarize', methods=['POST'])
def abs_summarize():
    text = request.form['text']
    if not text.strip():
        return render_template_string(HTML_TEMPLATE, summary="Error: No text provided.")

    summary = summarizer(text, max_length=100, min_length=10, do_sample=False)
    return render_template_string(HTML_TEMPLATE, summary=summary[0]['summary_text'])

# Start Ngrok and get a public URL
public_url = ngrok.connect(5000)
print(f"Public URL: {public_url}")

if __name__ == '__main__':
    app.run(port=5000)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Device set to use cpu


Public URL: NgrokTunnel: "https://7fc7-34-80-60-93.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [03/Apr/2025 16:17:53] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [03/Apr/2025 16:17:53] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [03/Apr/2025 16:18:30] "POST /ext_summarize HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [03/Apr/2025 16:18:55] "POST /abs_summarize HTTP/1.1" 200 -
